In [1]:
#Array packages
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4

from scipy.stats import kendalltau
import pymannkendall as mk

#plots
import matplotlib.pyplot as plt
import rioxarray as rio
import geopandas as gpd
from shapely.geometry import mapping

#Progress meter
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Directories
import os
import glob
import dask
#import h5netcdf
import scipy

import os
os.chdir(r"E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Research\1_Streamflow")
print(os.getcwd())

c:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Research\1_Streamflow


## Gauge Station

#### 1. Extracting the grids corresponding to gauge station

In [2]:
gauge_info=pd.read_csv('3_Data/Data_p/2_Station/gauge_info_p.csv')
data1=sorted(glob.glob(f'3_Data/Data_r/1_ILDAS/*.nc*'))
d_IL = xr.open_mfdataset(data1, combine='by_coords',engine='netcdf4', parallel=True,chunks={'time': 'auto'})


station=gauge_info['Station'].to_numpy()
date1=pd.date_range(start='1970',end='2021',freq='D')
Ildas_st=pd.DataFrame(index=date1, columns=station)


In [3]:
from tqdm import tqdm
for stn in tqdm(station):
    
    lat1=gauge_info[gauge_info['Station']==stn]['Latitude'].to_numpy()
    lon1=gauge_info[gauge_info['Station']==stn]["Longitude"].to_numpy()

    ds=d_IL['Streamflow_tavg'].sel(lat=lat1,lon=lon1,method='nearest').compute()

    Ildas_st.loc[date1,stn]=ds.sel(time=date1).values.reshape(-1)

Ildas_st_xr=xr.DataArray(np.array(Ildas_st),coords=[date1,station],dims=['time','station'],name='streamflow')
Ildas_st_xr = xr.Dataset({'streamflow': Ildas_st_xr})
Ildas_st_xr.to_netcdf(f'3_Data/Data_p/1_ILDAS/ILDAS_station_streamflow.nc') 

 90%|█████████ | 217/241 [2:21:13<15:55, 39.82s/it]  

### 2. Index calculation (seasonal)

In [2]:
### INDICES CALCULATION_DRY_WET period


## Season
ssn=['All','Pre-monsoon','Monsoon','Post-monsoon','Winter']
ssn_n=[range(1,13),[3,4,5],[6,7,8,9],[10,11],[12,1,2]]
ds_drt=[];ds_fld=[]
data=xr.open_mfdataset(f'3_Data/Data_p/1_ILDAS/ILDAS_station_streamflow.nc')


for s in tqdm(range(5)):

    ## drt
    drt_ssn=data['streamflow'].sel(time=data["time.month"].isin(ssn_n[s])).rolling(time=7).mean().resample(time='YE').min()
    drt_ssn=drt_ssn.expand_dims(season=[ssn[s]])
    ds_drt.append(drt_ssn)

    ## Flood
    fld_ssn=data['streamflow'].sel(time=data["time.month"].isin(ssn_n[s])).resample(time='YE').quantile(0.95, dim='time')
    fld_ssn=fld_ssn.expand_dims(season=[ssn[s]])
    ds_fld.append(fld_ssn)

    
drt=xr.concat(ds_drt,dim='season')
fld=xr.concat(ds_fld,dim='season').drop(['quantile'])

dry_wet_ssn = xr.Dataset({'drt_flow': drt, 'fld_flow': fld})
dry_wet_ssn.to_netcdf(r'3_Data/Data_p/1_ILDAS/output/ILDAS_station_indices.nc')




100%|██████████| 5/5 [00:04<00:00,  1.03it/s]
C:\Users\sstar\AppData\Local\Temp\ipykernel_2544\1528211280.py:25: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  fld=xr.concat(ds_fld,dim='season').drop(['quantile'])


## Gridwise ANALYSIS
Calculate drought flow (annual 7 days minimum flow) and flood flow

In [4]:
## Season
ssn=['Pre-monsoon','Monsoon','Post-monsoon','Winter']
ssn_n=[[3,4,5],[6,7,8,9],[10,11],[12,1,2]]
ds_drt=[];ds_fld=[]

data1=sorted(glob.glob(f'3_Data/Data_r/1_ILDAS/*.nc*'))
data = xr.open_mfdataset(data1, combine='by_coords',engine='netcdf4', parallel=True,chunks={'time': 'auto'})
data = data.sel(time=slice('1975','2022'))
data = data.chunk({'time': -1})

for s in tqdm(range(4)):

    ## drt
    drt_ssn=data['Streamflow_tavg'].sel(time=data["time.month"].isin(ssn_n[s])).rolling(time=7).mean().resample(time='YE').min().compute()
    drt_ssn=drt_ssn.expand_dims(season=[ssn[s]])
    ds_drt.append(drt_ssn)

    ## Flood
    fld_ssn=data['Streamflow_tavg'].sel(time=data["time.month"].isin(ssn_n[s])).resample(time='YE').quantile(0.95, dim='time').compute()
    fld_ssn=fld_ssn.expand_dims(season=[ssn[s]])
    ds_fld.append(fld_ssn)

    
drt=xr.concat(ds_drt,dim='season').drop(['spatial_ref'])
fld=xr.concat(ds_fld,dim='season').drop(['quantile'])

dry_wet_ssn = xr.Dataset({'drt_flow': drt, 'fld_flow': fld})
dry_wet_ssn.to_netcdf(r'3_Data/Data_p/1_ILDAS/output/ILDAS_indices.nc')



  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\xarray\core\indexing.py:1430: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
  0%|          | 0/4 [01:33<?, ?it/s]


MemoryError: Unable to allocate 128. MiB for an array with shape (143, 330, 355) and data type float64